In [1]:
#Setting up
%pip install requests

import pandas as pd
import numpy as np
import math
import random
import requests
import re

random.seed(2022)
np.random.seed(2022)

SENTIC_UI_API = "http://sentic.net/api/en/aL41u1Hf0mR.py"


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.comNote: you may need to restart the kernel to use updated packages.



In [2]:
#Baseline: Senticnet
class APIcall:
    def get_data(self, api, text):
        parameters = {"text": text}
        response = requests.get(f"{api}", parameters)
        if response.status_code == 200:
            # print("sucessfully fetched the data")
            # print(response.text)
            return response.text
        else:
            print(f"Hello, there's a {response.status_code} error with your request")

    def __init__(self):
        test_text = "senticnet is awesome"
        #self.get_data(api, test_text)

In [3]:
#Test whether autocall of API works
api_call = APIcall()
text = "I am fed up with this assignment"
response = api_call.get_data(SENTIC_UI_API, text)
temp = re.search("JustGage\({id:'ui',title:'(.+?)',label:", response)
start = temp.start()
start = start + 25
finish = start + 7

extracted_polarity = response[start:finish]
print(extracted_polarity)

POSITIV


In [9]:
#Read dataset
penn_df = pd.read_excel('Pennsylvania_with_faction_final.xlsx')
penn_df.head()

,Date,Content,Id,Username,replyCount,retweetCount,likeCount,quoteCount,conversationId,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,sentimentScore,subjective,sarcasm,faction,tweet_sequence
0,2022-10-10 23:25:02+00:00,Note to #JohnFetterman \nThis Neck-Warmer-Hood...,1.579614e+18,AutoCarbine556,0.0,0.0,1.0,0.0,1.579614e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,2022-10-10 23:01:21+00:00,#DrOz says #JohnFetterman's health makes him a...,1.579608e+18,robinstrasser,0.0,1.0,25.0,0.0,1.579608e+18,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,3.0
2,2022-10-10 21:15:50+00:00,"Hey, at this Fetterman-Oz debate, if they ask ...",1.579581e+18,rainbowbucs,1.0,0.0,1.0,0.0,1.579581e+18,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,4.0
3,2022-10-10 21:12:43+00:00,@NYMag @JohnFetterman @rtraister It's simple: ...,1.579581e+18,FischBytes,1.0,0.0,2.0,1.0,1.579412e+18,NaN,NaN,1.579413e+18,https://twitter.com/NYMag,"[User(username='NYMag', id=45564482, displayna...",NaN,NaN,NaN,NaN,5.0
4,2022-10-10 20:09:10+00:00,#WARNING VIEWER DISCRETION IS ADVISED. VIDEO S...,1.579565e+18,sn00pdad,0.0,0.0,0.0,0.0,1.579565e+18,NaN,NaN,NaN,NaN,"[User(username='MarkFusetti', id=7202495685044...",NaN,NaN,NaN,NaN,6.0


In [10]:
penn_df = penn_df[['Content', 'Id', 'sentimentScore', 'faction']].dropna()
penn_df.describe()

,Id,sentimentScore,faction
count,9.190000e+02,919.000000,919.000000
mean,1.571970e+18,0.566921,0.690968
std,5.987464e+15,0.495771,0.483085
min,1.557534e+18,0.000000,0.000000
25%,1.568371e+18,0.000000,0.000000
50%,1.574092e+18,1.000000,1.000000
75%,1.577282e+18,1.000000,1.000000
max,1.579608e+18,1.000000,2.000000


In [11]:
penn_df.head()

,Content,Id,sentimentScore,faction
1,#DrOz says #JohnFetterman's health makes him a...,1.579608e+18,1.0,1.0
2,"Hey, at this Fetterman-Oz debate, if they ask ...",1.579581e+18,1.0,1.0
5,"Missed this when originally posted, but fantas...",1.579557e+18,1.0,1.0
6,We need more #politicians like #JohnFetterman....,1.579549e+18,1.0,1.0
7,I just donated. PA is our best chance to add a...,1.579547e+18,1.0,1.0


In [12]:
def get_raw_sentiment(sentimentScore, faction):
    if (sentimentScore == faction):
        raw_sentiment = sentimentScore
    elif (faction == 2): 
        raw_sentiment = -1
    elif (sentimentScore != faction):
        raw_sentiment = 1 - sentimentScore
    return raw_sentiment


In [16]:
penn_df['rawSentiment'] = penn_df.apply(lambda x: get_raw_sentiment(x['sentimentScore'], x['faction']), axis=1)
penn_df.head()

,Content,Id,sentimentScore,faction,rawSentiment
1,#DrOz says #JohnFetterman's health makes him a...,1.579608e+18,1.0,1.0,1.0
2,"Hey, at this Fetterman-Oz debate, if they ask ...",1.579581e+18,1.0,1.0,1.0
5,"Missed this when originally posted, but fantas...",1.579557e+18,1.0,1.0,1.0
6,We need more #politicians like #JohnFetterman....,1.579549e+18,1.0,1.0,1.0
7,I just donated. PA is our best chance to add a...,1.579547e+18,1.0,1.0,1.0


In [17]:
#Function for extracting polarity from senticnet
#Returns a list that contains all the polarity in the same order
def extract_polarity_senticnet(text_list):
    result_list = []
    i=0
    for text in text_list: 
        response = api_call.get_data(SENTIC_UI_API, text)
        temp = re.search("JustGage\({id:'ui',title:'(.+?)',label:", response)
        start = temp.start()
        start = start + 25
        finish = start + 7
        extracted_polarity = response[start:finish]
        result_list.append(extracted_polarity)
        i += 1

    return result_list


In [18]:
#Get from senticnet
#penn_df_sentic = penn_df.iloc[:1000]
penn_df_sentic = penn_df
content_list = penn_df_sentic['Content'].tolist()
result_list = extract_polarity_senticnet(content_list)
penn_df_sentic['polarity'] = result_list
penn_df_sentic.head()

,Content,Id,sentimentScore,faction,rawSentiment,polarity
1,#DrOz says #JohnFetterman's health makes him a...,1.579608e+18,1.0,1.0,1.0,NEGATIV
2,"Hey, at this Fetterman-Oz debate, if they ask ...",1.579581e+18,1.0,1.0,1.0,NEGATIV
5,"Missed this when originally posted, but fantas...",1.579557e+18,1.0,1.0,1.0,POSITIV
6,We need more #politicians like #JohnFetterman....,1.579549e+18,1.0,1.0,1.0,POSITIV
7,I just donated. PA is our best chance to add a...,1.579547e+18,1.0,1.0,1.0,POSITIV


In [19]:
penn_df_sentic['polarity'].describe()

count         919
unique          3
top       POSITIV
freq          470
Name: polarity, dtype: object

In [21]:
penn_df_sentic['prediction'] = penn_df_sentic['polarity'].replace({'POSITIV':1,'NEGATIV':0, 'NEUTRAL':2})

In [22]:
from sklearn.metrics import classification_report
report = classification_report(penn_df_sentic.rawSentiment, penn_df_sentic.prediction)
print(report)

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         9
         0.0       0.35      0.43      0.39       293
         1.0       0.66      0.50      0.57       617
         2.0       0.00      0.00      0.00         0

    accuracy                           0.48       919
   macro avg       0.25      0.23      0.24       919
weighted avg       0.55      0.48      0.51       919



c:\Users\16680\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\16680\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\16680\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 